In [16]:
import nltk 
from nltk.stem import WordNetLemmatizer
import numpy as np
import json
import string
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

In [17]:
df =open('Layan.json').read()

In [18]:
data=json.loads(df)

In [19]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['hello', 'hi', 'hey', "what's up", 'anybody here', 'good day'],
   'responses': ['Hello!',
    'Hi there, how can I help you today?',
    'Feel free to ask me anything',
    'Ask me about Data Science!']},
  {'tag': 'Target User',
   'patterns': ["who's a Target user for the chatbot'",
    'for who they create you'],
   'responses': ['Beginners in the field',
    'First year students',
    'those who want to start in the field']},
  {'tag': 'thanks',
   'patterns': ['thank you',
    'thanks',
    'thanks for help',
    "that's helpful",
    'you are a greet help',
    'awesome',
    'very helpful'],
   'responses': ['Happy to help!',
    "You're welcome!",
    'Feel free to ask me again',
    'Anytime!',
    'My pleasure']},
  {'tag': 'nothing',
   'patterns': [],
   'responses': ['Please provide more context',
    'Not sure I understand',
    "Sorry, I can't understand you"]},
  {'tag': 'options',
   'patterns': ['How you could help me?

In [20]:
words = []
classes = []
data_x = [] 
data_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_x.append(pattern)
        data_y.append(intent["tag"])

    if intent["tag"] not in classes:
        classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))


In [21]:
import numpy as np
import random

# Your previous code
out_empty = [0] * len(classes)
training = []

for idx, doc in enumerate(data_x):
    bow = [] 
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        if word in text:
            bow.append(1)
        else:
            bow.append(0)
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    
    training.append((bow, output_row))

random.shuffle(training)

train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

train_x = np.array(train_x)
train_y = np.array(train_y)


In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout



model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))  # Assuming this is a multi-class classification task

# Use the legacy optimizer
adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])
print(model.summary())



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               20608     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 59)                3835      
                                                                 
Total params: 32699 (127.73 KB)
Trainable params: 32699 (127.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [27]:
model.fit(x=train_x, y=train_y,epochs=150,verbose=1)

Epoch 1/150
6/6 [==============================] - 1s 5ms/step - loss: 4.1155 - accuracy: 0.0213
Epoch 2/150
6/6 [==============================] - 0s 4ms/step - loss: 3.9207 - accuracy: 0.0691
Epoch 3/150
6/6 [==============================] - 0s 3ms/step - loss: 3.7213 - accuracy: 0.1543
Epoch 4/150
6/6 [==============================] - 0s 6ms/step - loss: 3.4548 - accuracy: 0.1383
Epoch 5/150
6/6 [==============================] - 0s 4ms/step - loss: 3.1556 - accuracy: 0.1862
Epoch 6/150
6/6 [==============================] - 0s 6ms/step - loss: 2.7217 - accuracy: 0.3351
Epoch 7/150
6/6 [==============================] - 0s 5ms/step - loss: 2.5085 - accuracy: 0.3351
Epoch 8/150
6/6 [==============================] - 0s 5ms/step - loss: 2.1673 - accuracy: 0.4309
Epoch 9/150
6/6 [==============================] - 0s 12ms/step - loss: 1.8438 - accuracy: 0.4415
Epoch 10/150
6/6 [==============================] - 0s 5ms/step - loss: 1.7873 - accuracy: 0.4787
Epoch 11/150
6/6 [==========

In [30]:
#Preprocessing the Input
def clean_text(text):
    tokens =nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return tokens
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word ==w:
                bow[idx] =1
    return np.array(bow)
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list)==0:
        result="Soory! I don't unerstand"
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        for i in list_of_intents:
            if i["tag"]==tag:
                result=random.choice(i["responses"])
                break
    return result

In [11]:
import streamlit as st

# Assuming pred_class and get_response are defined somewhere in your code
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list)==0:
        result="Soory! I don't unerstand"
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        for i in list_of_intents:
            if i["tag"]==tag:
                result=random.choice(i["responses"])
                break
    return result

def main():
    st.title("Chatbot with Streamlit")
    st.write("Press 0 if you don't want to chat with the chatbot")

    while True:
        message = st.text_input("You: ")
        
        if message == "0":
            break
        
        intents = pred_class(text, vocab, labels)
        result = get_response(intents, data)

        st.text(f"Chatbot: {result}")



In [4]:
pip install altair==4.1.0


  Attempting uninstall: altair
    Found existing installation: altair 5.2.0
    Uninstalling altair-5.2.0:
      Successfully uninstalled altair-5.2.0



In [5]:
pip install pandas altair plotly


In [37]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install --upgrade jinja2


In [35]:
pip install --upgrade pandas streamlit


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\USER\\anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


In [31]:
print("Press 0 if you don't want to chat with chatbot")
while True:
    message = input("")
    if message == "0":
        break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)
    

Press 0 if you don't want to chat with chatbot
hi
1/1 [==============================] - 0s 19ms/step
Hi there, how can I help you today?
tell me about data analysis
1/1 [==============================] - 0s 21ms/step
Data analysis is the process of inspecting, cleaning, transforming, and modeling data with the goal of discovering useful information, drawing conclusions, and supporting decision-making.
ide
1/1 [==============================] - 0s 31ms/step
Jupyter Notebooks: https://www.anaconda.com/
ide
1/1 [==============================] - 0s 32ms/step
Jupyter Notebooks: https://www.anaconda.com/
bye
1/1 [==============================] - 0s 39ms/step
Hello!
thank you
1/1 [==============================] - 0s 46ms/step
Anytime!
0


In [32]:
pip install streamlit

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
spyder 5.1.5 requires pyqt5<5.13, but you have pyqt5 5.15.4 which is incompatible.



  Attempting uninstall: pygments
    Found existing installation: Pygments 2.10.0
    Uninstalling Pygments-2.10.0:
      Successfully uninstalled Pygments-2.10.0


In [2]:
pip install jinja2==3.1.2


  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.0.0
    Uninstalling Jinja2-3.0.0:
      Successfully uninstalled Jinja2-3.0.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
spacy 3.5.3 requires thinc<8.2.0,>=8.1.8, but you have thinc 8.2.1 which is incompatible.
flask-sqlalchemy 3.0.0 requires SQLAlchemy>=1.4.18, but you have sqlalchemy 1.2.19 which is incompatible.
cookiecutter 1.7.2 requires Jinja2<3.0.0, but you have jinja2 3.1.2 which is incompatible.
cookiecutter 1.7.2 requires MarkupSafe<2.0.0, but you have markupsafe 2.1.1 which is incompatible.
